# Opdracht Autoencoders


Autoencoders zijn neurale netwerken die getraind worden om hun eigen input te reconstrueren. De toepassingen die mogelijk gemaakt worden door autoencoders variëren van compressie, denoising, reconstructie van afbeeldingen, unsupervised pre-training tot en met one-class classificatie, resolution upscaling, recommendation systems, anomaly detection en image segmentation.

Deze opdracht bestaat uit 3 deelopdrachten:
1. Anomaly detection
2. Denoising 
3. Image segmentation

## 1. Fraude detectie

Een anomalie is een datapunt dat qua statistische eigenschappen sterk afwijkt van de meerderheid van de datapunten. Zo kunnen ook frauduleuze banktransacties aanzien worden als anomalieën.
Een van de grootste uitdagingen bij het detecteren van anomaliën is de ongebalanceerdheid van de data. Deze anomalieën komen per definitie vrij uitzonderlijk voor. 

Een mogelijke benadering voor het detecteren van anomalieën is leren van de distributie van normale datapunten, in dit geval normale banktransacties. Vervolgens kan je nieuwe datapunten classificeren als anomalieën wanneer ze volgens de geleerde distributie heel onwaarschijnlijk zijn.

Autoencoders kunnen de distributie van de normale datapunten leren door deze normale datapunten te leren reconstrueren. De getrainde autoencoder zal vervolgens moeite hebben om anamalieën te reconstrueren met als resultaat een hoge reconstructie-error, bijvoorbeeld de MSE = Mean Squared Error. https://keras.io/losses/

𝑀𝑆𝐸=𝑖𝑛∑𝑛𝑖=1(𝑌−𝑌̂ )2

Een transactie kan dus als frauduleus verondersteld worden wanneer de reconstructie-error boven een zekere threshold uitsteekt. 
Men spreekt van een one-class classifier.


Gegeven is een dataset met banktransacties. Bepaalde features zijn geanonimiseerd (aangeduid met V). Andere features duiden het bedrag en het tijdstip aan. Alle transacties kregen het label 1 (frauduleus) of 0 (normale transactie).



Doorloop de volgende stappen:

- Controleer in welke mate de dataset gebalanceerd is.

- Preprocessing.

- Opbouwen van een training set en test set.

- Train de autoencoder.

- Gebruik de classificatie-error om een transactie te classificeren als normaal of frauduleus.

- Test de autoencoder op de test set.

- Voer hyperparameter tuning uit op het neuraal netwerk, maar zoek ook een gepaste waarde voor de threshold op de reconstructie error. 

- Beantwoord de vragen.


In [10]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

import matplotlib.image as mpimg
from skimage.io import imread, imshow
from skimage.color import rgb2gray
from skimage import data, color, io, filters, morphology,transform, exposure, feature, util
from scipy import ndimage


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, BatchNormalization,concatenate
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image
from tensorflow.keras import regularizers


# Voor GPU support
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)





In [8]:
# Inlezen van de dataset

dataset = pd.read_csv('creditcard.csv')
dataset.head()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


### Analyse van de dataset

- Controleer in welke mate de dataset ongebalanceerd is. Maak daarvoor een histogram van de klasselabels. Een geschikte plot is de seaborn countplot (https://seaborn.pydata.org/generated/seaborn.countplot.html).


In [4]:
# Analyse van de dataset


### Preprocessing

- Kuis de dataset op. Verwijder de irrelevante features.
- Splits op in een trainig set en test set. Zorg ervoor dat je een gebalanceerde test set bekomt. 

In [5]:
# Preprocessing


### Ontwerpen, trainen en hyperparameter tuning v.d. autoencoder

- Ontwerp en train een autoencoder. Gebruik de MSE (Mean Squared Error) loss function. Kies zelf de grootte van het neurale netwerk. Deze kan je later bij de hyperparametertuning nog aanpassen.
- Kies een threshold voor de MSE waarmee je beslist of een transactie al dan niet frauduleus is.
- Gebruik de test set om de autoencoder te evalueren. Gebruik hiervoor de accuraatheid, confusion matrix, recall, precision, f1-score en de ROC.
- Voer hyperparameter tuning uit om de performantie te verhogen. Zoek de optimale MSE threshold. Visualeer welke punten boven de threshold liggen en welke eronder. Gebruik kleur om de verschillende klasses aan te duiden.
- Stel dat de bank zoveel mogelijk false-negatives (frauduleuze transacties die als normaal werden geclassificeerd) wil vermijden. Welke aanpassingen zou je daarvoor kunnen doen? Test deze.

### Ontwerp en training van de autoencoder


In [ ]:
# Testen van de autoencoder


In [6]:
# Hyperparameter tuning. Zoek naar de optimale waarde voor de MSE-threshold


In [ ]:
# Vermijden van false negatives.


### Beantwoord de onderstaande vragen

-  Welk type autoencoder is het meest geschikt (undercomplete, overcomplete, convolutional, ...)? Waarom?

- Zou het zinvol zijn om de categorical cross-entropy te gebruiken als loss functie?

- Plot een histogram van de MSE waarden op de training set. Hoe zijn deze verdeeld? Kan je hieruit een schatting maken van waar de threshold zal komen te liggen?

ANTWOORDEN:



## 2. Reconstructie van partially occluded faces

Partially occluded faces zijn gezichten die gedeeltelijk bedekt zijn, bijvoorbeeld door een bril, sjaal, hoofddeksel, enzoverder.
Via een autoencoder zullen we proberen te achterhalen wat er zich achter die bedekking bevindt en een poging doen het bedekte gedeelte te reconstrueren.
Om dit te realiseren moeten we over een trainig set beschikken die bestaat uit zowel bedekte als niet-bedekte afbeeldingen van hetzelfde gezicht.
Deze training set gaan we zelf genereren vertrekkende vanaf een bestaande gezichtendataset te vinden in de map 'Faces'. Het bedekken van het gezicht gebeurt door op een willekeurige plaats in de afbeelding een wit vierkant aan te brengen. Een voorbeeld vind je hieronder:

![Partially occluded face](./NotebookImages/occlusion_example.png)

Zorg er zeker voor dat de trainingset voldoende uitgebreid is. Je kan dezelfde afbeeldingen meerdere malen gebruiken om bedekking op aan te brengen. Zo creëer je een training set die in aantal afbeeldingen een veelvoud is van de originele dataset.

- Probeer verschillende types autoencoders uit (convolutional, undercomplete, overcomplete, ...). Varieer tevens het aantal en de grootte van de layers.
- De afbeeldingen kunnen best in grootte gereduceerd worden om het berekenbaar te houden.
- Probeer verschillende loss functies en optimizers.
- Argumenteer en beschrijf telkens de resultaten en bevindingen van een gebruikte autoencoder

## 3. Image segmentation

Autoencoders kunnen ook getraind worden om image segmentation uit te voeren. Segmentatie betekent dat je specifieke objecten in de afbeelding gaat afzonderen van de rest van de afbeelding. In de praktijk komt dit meestal neer op het inkleuren het desbetreffende object in de afbeelding. Image segmentation vindt onderandere zijn toepassing bij self-driving cars, video surveillance en medical imaging. Bij deze opdracht is het de bedoeling om longen te segmenteren uit X-ray afbeeldingen.

Foto's van de longen zijn te vinden in de map Lung_images. 

De labels zijn in dit geval mask afbeeldingen die de exacte locatie van de longen aanduiden. Deze masks bevinden zich in de map Lung_masks.

Ga bij deze opdracht als volgt te werk:

- Lees de afbeeldingen in en schaal deze allemaal naar dezelfde grootte. Bijvoorbeeld naar 400x400 pixels. Lagere resolutie mag ook als je merkt dat de systeemvereisten niet volstaan. Voor deze toepassing mag alles naar grijswaarden worden omgezet.

- Maak een training set en test set aan. Stop een 20-tal afbeeldingen in de test set.

- Train een convolutional autoencoder op de training set. De output layer moet dezelfde dimensie hebben als de mask images. De autoencoder wordt namelijk getraind om op basis van een long-scan een mask te genereren die zo goed mogelijk de ground truth mask benadert. Dit vereist echter een aangepaste loss functie om de autoencoder mee te trainen. Cross-entropy gebaseerde loss functies zijn hiervoor niet altijd geschikt. In de praktijk wordt dikwijls de dice coefficient loss gebruikt. Deze kijkt naar de overlap tussen twee data samples. In dit geval hoeveel de door de autoencoder gegeneerde mask overlapt met de ground truth mask. Deze dice loss moet je zelf als custom loss meegeven met het neuraal netwerk.

```Python
def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2.*intersection + smooth)/(K.sum(K.square(y_true),-1)+ K.sum(K.square(y_pred),-1) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

```   

- Test de autoencoder op de afbeeldingen uit de test set. Visualiseer de gegeneerde mask en vergelijk met de werkelijke mask. Probeer de segmentatie te verbeteren via hyperparameter tuning. Gebruik hiervoor naast visuele inspectie ook de dice coefficient als metric.

- U-net is een veelgebruikt neuraal netwerk voor medical image segmentation. Een voorbeeld hiervan kan je vinden op https://www.kaggle.com/keegil/keras-u-net-starter-lb-0-277. Implementeer, train en test dit netwerk. Gebruik ook eens de dice coefficient loss als custom loss functie. Bespreek de resultaten.
   

In [11]:
# Dice coefficient loss

def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

In [12]:
# Uitwerking image segmentation
